<a href="https://colab.research.google.com/github/eorellan693/DAEN500_F2020/blob/master/VerticalSeparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data pre-processing will happen here. 
Doing an update. 
---
Will this be saved?

In [ ]:
#Installing the required libraries
!pip install geopy
!pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=797e21ea1594e497a30d6636cb81c021a401df648985622ab28a870dca641136
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


In [ ]:
#Importing the required libraries
import pandas as pd
import pandasql as ps
# from pyspark.sql import SparkSession
import os
import sys
import re
from datetime import datetime
from geopy.distance import geodesic as GD
from math import radians, cos, sin, asin, sqrt

In [ ]:
#Upload the data file after pre-processing (only the selected attribues and only above level 240)
from google.colab import files
uploadedfile = files.upload()


Saving airspaceData.csv to airspaceData.csv


In [ ]:
import io
airspaceDF = pd.read_csv(io.BytesIO(uploadedfile['airspaceData.csv']))

In [ ]:
print(airspaceDF)

                        DateTime  Hour  Minute  Second  microSecond  \
0        2021-12-24 23:59:59.992    23      59      59       992000   
1        2021-12-24 20:54:11.141    20      54      11       141000   
2        2021-12-24 12:36:02.773    12      36       2       773000   
3        2021-12-24 04:13:17.836     4      13      17       836000   
4        2021-12-24 23:59:59.984    23      59      59       984000   
...                          ...   ...     ...     ...          ...   
3303983  2021-12-24 12:36:02.859    12      36       2       859000   
3303984  2021-12-24 12:36:02.852    12      36       2       852000   
3303985  2021-12-24 12:36:02.805    12      36       2       805000   
3303986  2021-12-24 12:36:02.805    12      36       2       805000   
3303987  2021-12-24 12:36:02.789    12      36       2       789000   

          Latitude   Longitude   Height TargetID  SelectedHeading  \
0        21.325208 -158.058116   2200.0   UAL253        78.750000   
1        

In [ ]:
#Some column names may be imported with leading spaces. This will remove that
airspaceDF.columns = [x.strip() for x in airspaceDF.columns]

print(airspaceDF.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3303988 entries, 0 to 3303987
Data columns (total 12 columns):
 #   Column           Dtype  
---  ------           -----  
 0   DateTime         object 
 1   Hour             int64  
 2   Minute           int64  
 3   Second           int64  
 4   microSecond      int64  
 5   Latitude         float64
 6   Longitude        float64
 7   Height           float64
 8   TargetID         object 
 9   SelectedHeading  float64
 10  TargetAddress    object 
 11  TrackNumber      float64
dtypes: float64(5), int64(4), object(3)
memory usage: 302.5+ MB
None


In [ ]:
#create SQL query for flights between the start and end time
sql1 = "SELECT * FROM airspaceDF WHERE Hour = 23 and Minute = 59 ORDER BY TargetID, Second"

#Run query and store results
records2359 = ps.sqldf(sql1, locals())

In [ ]:
#Export the file with just the 23:59 data to make it easier to pick up where we left off
from google.colab import files

records2359.to_csv('records2359_1224.csv', encoding = 'utf-8-sig') 
files.download('records2359_1224.csv')

**START HERE WITH records2359_1224.csv FILE**

In [ ]:
#Upload the file with the 23:59 data and save it as a dataframe named records2359

#Upload the records2359_1224.csv data file
from google.colab import files
uploadedfile = files.upload()

import io
records2359 = pd.read_csv(io.BytesIO(uploadedfile['records2359_1224.csv']))

Saving records2359_1224.csv to records2359_1224.csv


In [ ]:
#Remove flights without a flight name or altitude
records2359 = records2359.dropna(subset=['TargetID'])
records2359 = records2359.dropna(subset=['Height'])

records2359.head(5)

,Unnamed: 0,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,Height,TargetID,SelectedHeading,TargetAddress,TrackNumber
16,16,2021-12-24 23:59:01.875,23,59,1,875000,21.085836,-157.526245,15400.0,AAH24,107.578125,A32F17,1234.0
17,17,2021-12-24 23:59:06.750,23,59,6,750000,21.081926,-157.517598,15400.0,AAH24,107.578125,A32F17,1234.0
18,18,2021-12-24 23:59:10.875,23,59,10,875000,21.078644,-157.510337,15400.0,AAH24,107.578125,A32F17,1234.0
19,19,2021-12-24 23:59:15.102,23,59,15,102000,21.075211,-157.502747,15425.0,AAH24,107.578125,A32F17,1234.0
20,20,2021-12-24 23:59:19.695,23,59,19,695000,21.071594,-157.494807,15400.0,AAH24,107.578125,A32F17,1234.0


In [ ]:
#Get the first entry for this minute of time
sql2 = "SELECT *, min(Second) FROM records2359 GROUP BY TargetID"

#Run query and store results
firstEntryDF = ps.sqldf(sql2, locals())
del firstEntryDF['min(Second)']

firstEntryDF.head(5)

,Unnamed: 0,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,Height,TargetID,SelectedHeading,TargetAddress,TrackNumber
0,16,2021-12-24 23:59:01.875,23,59,1,875000,21.085836,-157.526245,15400.0,AAH24,107.578125,A32F17,1234.0
1,30,2021-12-24 23:59:01.508,23,59,1,508000,19.976543,-155.128401,2625.0,AAH504,130.078125,A323F2,1272.0
2,43,2021-12-24 23:59:03.109,23,59,3,109000,25.290344,-147.790578,38775.0,AAL119,232.031250,AB6AFA,1024.0
3,56,2021-12-24 23:59:02.273,23,59,2,273000,42.300705,-132.887941,38200.0,AAL128,99.843750,AB7613,1024.0
4,69,2021-12-24 23:59:01.438,23,59,1,438000,33.262985,-123.632941,30500.0,AAL143,0.000000,A4FBAC,1024.0


In [ ]:
#Create tables to store proximity
sql3 = "SELECT * FROM firstEntryDF ORDER BY Latitude"
sql4 = "SELECT * FROM firstEntryDF ORDER BY Longitude"

#Run query and store results
LatitudeOrderDF = ps.sqldf(sql3, locals())
LongitudeOrderDF = ps.sqldf(sql4, locals())


In [ ]:
LatitudeOrderDF.head()

,Unnamed: 0,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,Height,TargetID,SelectedHeading,TargetAddress,TrackNumber
0,1100,2021-12-24 23:59:04.047,23,59,4,47000,0.190538,-175.769980,37350.0,FDX77,210.937500,ABE1EB,1024.0
1,1800,2021-12-24 23:59:00.625,23,59,0,625000,6.845460,-163.240356,36000.0,MGE600,NaN,AC3C99,1484.0
2,659,2021-12-24 23:59:03.977,23,59,3,977000,11.171885,178.954757,37225.0,ATN440,51.328125,AA21BF,1024.0
3,1366,2021-12-24 23:59:01.453,23,59,1,453000,11.610489,-166.661035,42300.0,HAL451,NaN,A45233,1024.0
4,2578,2021-12-24 23:59:04.836,23,59,4,836000,12.478409,142.616035,40650.0,SIA8890,258.750000,76B44A,1997.0


In [ ]:
LongitudeOrderDF.head()

,Unnamed: 0,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,Height,TargetID,SelectedHeading,TargetAddress,TrackNumber
0,1787,2021-12-24 23:59:03.648,23,59,3,648000,17.969330,-176.990517,38025.0,MGE300,NaN,A9228A,1088.0
1,1100,2021-12-24 23:59:04.047,23,59,4,47000,0.190538,-175.769980,37350.0,FDX77,210.937500,ABE1EB,1024.0
2,3382,2021-12-24 23:59:02.883,23,59,2,883000,33.536041,-175.560315,40950.0,UAL903,288.281250,AA9321,1488.0
3,3368,2021-12-24 23:59:00.430,23,59,0,430000,49.627487,-174.973249,34575.0,UAL893,265.078125,A31A73,1024.0
4,257,2021-12-24 23:59:00.109,23,59,0,109000,47.560684,-172.337723,40800.0,AAL9726,277.031250,AB5257,1024.0


In [ ]:
# Implement the formula below
def distance_d(point0,pointX):
# The function "radians" is found in the math module, It's also used to convert radians to degrees.  
 LoA = radians(point0[1])  
 LoB = radians(pointX[1])
 LaA=  radians(point0[0])  
 LaB = radians(pointX[0]) 
# The "Haversine formula" is used.
 D_Lo = LoB - LoA 
 D_La = LaB - LaA 
 P = sin(D_La / 2)**2 + cos(LaA) * cos(LaB) * sin(D_Lo / 2)**2  
   
 Q = 2 * asin(sqrt(P))   
 # The earth's radius in kilometers.
 R_km = 6371  
 
 # Change the kilometer to  nautical miles
 R_nm = R_km*0.539956803

# Then we'll compute the outcome.
 return(Q * R_nm)
 


In [ ]:
#Calculate the distance of the points closest to each other by longitidue and latitude



longpoint_a = []
longpoint_b = []
longdistance_ab = []

latpoint_a = []
latpoint_b = []
latdistance_ab = []

for a in LongitudeOrderDF.index:   
    for n in range(1,len(LongitudeOrderDF)): 
    # check distance with next flights until found distance >25, then out of the inner loop
        b = a+n
        if b < len(LongitudeOrderDF):
            point0 = LongitudeOrderDF.loc[a,'Latitude'], LongitudeOrderDF.loc[a,'Longitude']
            pointX = LongitudeOrderDF.loc[b,'Latitude'], LongitudeOrderDF.loc[b,'Longitude']    
            distance = distance_d(point0,pointX) #***change distance calculation***
            if distance <= 25: #***change limit closet distance***
                longpoint_a.append(a)
                longpoint_b.append(b)
                longdistance_ab.append(distance)
            else:
                break
print("Points closest to each other by longitiude","\n",longpoint_a,"\n",longpoint_b,"\n", longdistance_ab,"\n")

for a in LatitudeOrderDF.index:   
    for n in range(1,len(LatitudeOrderDF)): 
    # check distance with next flights until found distance > 5, then out of the inner loop
        b = a+n
        if b < len(LatitudeOrderDF):
            point0 = LatitudeOrderDF.loc[a,'Latitude'], LatitudeOrderDF.loc[a,'Longitude']
            pointX = LatitudeOrderDF.loc[b,'Latitude'], LatitudeOrderDF.loc[b,'Longitude']    
            distance = distance_d(point0,pointX)  #***change distance calculation***
            if distance <= 25: #***change limit closet distance***
                latpoint_a.append(a)
                latpoint_b.append(b)
                latdistance_ab.append(distance)
            else:
                break
print("Points closest to each other by latitiude","\n",latpoint_a,"\n", latpoint_b,"\n", latdistance_ab)

Points closest to each other by longitiude 
 [6, 14, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 30, 30, 30, 30, 31, 31, 31, 32, 32, 33, 33, 34, 36, 36, 36, 36, 37, 37, 37, 37, 37, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 40, 40, 40, 40, 40, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 44, 44, 44, 44, 44, 44, 44, 44, 44, 45, 45, 45, 45, 45, 45, 45, 45, 46, 46, 46, 46, 46, 46, 46, 47, 47, 47, 47, 47, 47, 48, 48, 48, 48, 48, 49, 49, 49, 49, 50, 50, 50, 51, 51, 52, 55, 55, 55, 56, 56, 57, 60, 60, 60, 60, 61, 61, 61, 61, 62, 62, 62, 62, 62, 63, 63, 63, 63, 64, 64, 64, 65, 65, 65, 66,

In [ ]:
# Create function to select, merge and add the values from analyzing Longitude and Latitude
def newDF(OrderDF,x,y,d):
    """DF is Long/LatitudeOrderDF
       x = long/latpoint_a
       y = long/latpoint_b
       d = long/latdistance_ab"""
    # select rows that index is in list 'point_a', 'point_b'
    A = OrderDF.loc[x,['DateTime','Hour','Minute','Second','microSecond','Latitude','Longitude','Height',
                             'TargetID', 'SelectedHeading', 'TargetAddress']]
    B = OrderDF.loc[y,['DateTime','Hour','Minute','Second','microSecond','Latitude','Longitude','Height',
                             'TargetID', 'SelectedHeading', 'TargetAddress']]
    # Join 2 tables by the "TargetID" of point a (for the uniquness)
    OrderResult = pd.merge(A.reset_index(drop=True),B.reset_index(drop=True),left_index=True, right_index=True)
    # add distance column
    OrderResult['Distance'] = d
    return OrderResult

In [ ]:
# Apply function to select and merge data frame
LongOrderResult = newDF(LongitudeOrderDF,longpoint_a, longpoint_b,longdistance_ab)
LatOrderResult = newDF(LatitudeOrderDF,latpoint_a, latpoint_b,latdistance_ab)

In [ ]:
# Concatenate results from order by longitude and latitude
Resultsdf = pd.concat([LongOrderResult,LatOrderResult]).reset_index(drop=True)

# Show result
Resultsdf

,DateTime_x,Hour_x,Minute_x,Second_x,microSecond_x,Latitude_x,Longitude_x,Height_x,TargetID_x,SelectedHeading_x,...,Minute_y,Second_y,microSecond_y,Latitude_y,Longitude_y,Height_y,TargetID_y,SelectedHeading_y,TargetAddress_y,Distance
0,2021-12-24 23:59:02.734,23,59,2,734000,28.675294,-169.513442,37525.0,KAL054,296.015625,...,59,2,508000,28.527740,-169.294140,32325.0,JAL783,296.718750,86220C,14.564467
1,2021-12-24 23:59:02.086,23,59,2,86000,24.062439,-163.081970,39550.0,HAL459,NaN,...,59,2,773000,24.034607,-163.008677,41450.0,JAL8781,295.312500,86D996,4.352175
2,2021-12-24 23:59:10.336,23,59,10,336000,22.101746,-159.699656,3675.0,N11QP,NaN,...,59,1,719000,22.160156,-159.696317,2450.0,N11QN,NaN,A02B45,3.511916
3,2021-12-24 23:59:10.336,23,59,10,336000,22.101746,-159.699656,3675.0,N11QP,NaN,...,59,0,234000,22.160913,-159.664764,1800.0,N11QC,NaN,A02B3B,4.047942
4,2021-12-24 23:59:10.336,23,59,10,336000,22.101746,-159.699656,3675.0,N11QP,NaN,...,59,23,938000,22.067064,-159.643707,2600.0,N11QW,NaN,A02B4D,3.745012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,2021-12-24 23:59:03.836,23,59,3,836000,30.302062,-143.224915,38825.0,UAL384,229.218750,...,59,4,477000,30.508942,-142.826574,36875.0,HAL55,NaN,A1A546,24.078298
511,2021-12-24 23:59:04.477,23,59,4,477000,30.508942,-142.826574,36875.0,HAL55,NaN,...,59,4,117000,30.668381,-142.403977,37825.0,SWA2040,231.328125,AB5BB1,23.847714
512,2021-12-24 23:59:04.875,23,59,4,875000,34.055336,-134.166470,32475.0,DAL330,258.046875,...,59,1,789000,34.152816,-133.993607,36550.0,NCR149,234.843750,A9C52E,10.397546
513,2021-12-24 23:59:04.000,23,59,4,0,43.081100,-129.961108,32600.0,DAL565,213.750000,...,59,2,375000,43.141821,-130.404641,31700.0,ANA9643,281.953125,869210,19.779402


In [ ]:
# Delete duplicate pairs of TargetID x and y
ResultsDF = Resultsdf.drop_duplicates(subset = ['TargetID_x', 'TargetID_y','Distance'],keep = 'last').reset_index(drop = True)
ResultsDF

,DateTime_x,Hour_x,Minute_x,Second_x,microSecond_x,Latitude_x,Longitude_x,Height_x,TargetID_x,SelectedHeading_x,...,Minute_y,Second_y,microSecond_y,Latitude_y,Longitude_y,Height_y,TargetID_y,SelectedHeading_y,TargetAddress_y,Distance
0,2021-12-24 23:59:02.734,23,59,2,734000,28.675294,-169.513442,37525.0,KAL054,296.015625,...,59,2,508000,28.527740,-169.294140,32325.0,JAL783,296.718750,86220C,14.564467
1,2021-12-24 23:59:02.086,23,59,2,86000,24.062439,-163.081970,39550.0,HAL459,NaN,...,59,2,773000,24.034607,-163.008677,41450.0,JAL8781,295.312500,86D996,4.352175
2,2021-12-24 23:59:10.336,23,59,10,336000,22.101746,-159.699656,3675.0,N11QP,NaN,...,59,23,938000,22.067064,-159.643707,2600.0,N11QW,NaN,A02B4D,3.745012
3,2021-12-24 23:59:10.336,23,59,10,336000,22.101746,-159.699656,3675.0,N11QP,NaN,...,59,14,711000,22.054774,-159.530691,3375.0,N907AM,NaN,AC8939,9.814780
4,2021-12-24 23:59:10.336,23,59,10,336000,22.101746,-159.699656,3675.0,N11QP,NaN,...,59,1,234000,21.955398,-159.347651,750.0,N633JH,NaN,A849B1,21.471859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,2021-12-24 23:59:03.836,23,59,3,836000,30.302062,-143.224915,38825.0,UAL384,229.218750,...,59,4,477000,30.508942,-142.826574,36875.0,HAL55,NaN,A1A546,24.078298
443,2021-12-24 23:59:04.477,23,59,4,477000,30.508942,-142.826574,36875.0,HAL55,NaN,...,59,4,117000,30.668381,-142.403977,37825.0,SWA2040,231.328125,AB5BB1,23.847714
444,2021-12-24 23:59:04.875,23,59,4,875000,34.055336,-134.166470,32475.0,DAL330,258.046875,...,59,1,789000,34.152816,-133.993607,36550.0,NCR149,234.843750,A9C52E,10.397546
445,2021-12-24 23:59:04.000,23,59,4,0,43.081100,-129.961108,32600.0,DAL565,213.750000,...,59,2,375000,43.141821,-130.404641,31700.0,ANA9643,281.953125,869210,19.779402


In [ ]:
heightDifference = []
potentialLoss = []
potentialLoss1000 = []
potentialLoss400 = []

counter = 0

while counter < len(ResultsDF):
  difference = abs((ResultsDF['Height_x'][counter]) - (ResultsDF['Height_y'][counter]))
  heightDifference.append(difference)

  if difference <= 1000:
    potentialLoss.append('True')
  else:
    potentialLoss.append('False')

  counter = counter + 1

print(heightDifference)
print(potentialLoss)

[5200.0, 1900.0, 1075.0, 300.0, 2925.0, 2125.0, 3525.0, 3050.0, 150.0, 925.0, 875.0, 1700.0, 900.0, 2300.0, 1825.0, 800.0, 1575.0, 225.0, 1050.0, 250.0, 1650.0, 1175.0, 775.0, 1850.0, 1050.0, 2450.0, 1975.0, 2400.0, 600.0, 225.0, 575.0, 825.0, 350.0, 2200.0, 400.0, 425.0, 375.0, 1025.0, 550.0, 1450.0, 350.0, 1175.0, 375.0, 1775.0, 1300.0, 1650.0, 150.0, 975.0, 175.0, 1575.0, 1100.0, 2625.0, 1825.0, 3225.0, 2750.0, 800.0, 825.0, 25.0, 1425.0, 950.0, 2600.0, 600.0, 125.0, 3425.0, 1400.0, 925.0, 2625.0, 475.0, 4025.0, 3550.0, 6675.0, 2825.0, 350.0, 2450.0, 2475.0, 375.0, 6875.0, 1275.0, 4325.0, 2100.0, 4400.0, 3750.0, 6800.0, 4975.0, 4775.0, 5800.0, 2775.0, 6150.0, 6800.0, 6800.0, 6525.0, 6500.0, 1650.0, 4700.0, 2875.0, 2675.0, 3700.0, 675.0, 4700.0, 4700.0, 4425.0, 4350.0, 650.0, 8150.0, 11200.0, 9375.0, 9175.0, 10200.0, 3050.0, 1225.0, 1025.0, 975.0, 3050.0, 2700.0, 2300.0, 1825.0, 2025.0, 4025.0, 0.0, 350.0, 5350.0, 1025.0, 2000.0, 1375.0, 2025.0, 2025.0, 1750.0, 1675.0, 3325.0, 275.0,

In [ ]:
ResultsDF['HeightDifference'] = heightDifference
ResultsDF['PotentialLoss'] = potentialLoss

print(ResultsDF)

                  DateTime_x  Hour_x  Minute_x  Second_x  microSecond_x  \
0    2021-12-24 23:59:02.734      23        59         2         734000   
1    2021-12-24 23:59:02.086      23        59         2          86000   
2    2021-12-24 23:59:10.336      23        59        10         336000   
3    2021-12-24 23:59:10.336      23        59        10         336000   
4    2021-12-24 23:59:10.336      23        59        10         336000   
..                       ...     ...       ...       ...            ...   
442  2021-12-24 23:59:03.836      23        59         3         836000   
443  2021-12-24 23:59:04.477      23        59         4         477000   
444  2021-12-24 23:59:04.875      23        59         4         875000   
445  2021-12-24 23:59:04.000      23        59         4              0   
446  2021-12-24 23:59:02.570      23        59         2         570000   

     Latitude_x  Longitude_x  Height_x TargetID_x  SelectedHeading_x  ...  \
0     28.675294  -169.

In [ ]:
ResultsDF.to_csv('results2359.csv', encoding = 'utf-8-sig') 
files.download('results2359.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Get the first entry for this minute of time
sql6 = "SELECT * FROM Resultsdf WHERE PotentialLoss = 'True' "

#Run query and store results
LossCandidates = ps.sqldf(sql6, locals())

PandaSQLException: ignored

In [ ]:
print(LossCandidates)

    remove1 DateTime1  Hour1  Minute1  Second1  microSecond1  Latitude1  \
0      1922   59:01.7     23       59        1        719000  22.160156   
1      2262   59:14.4     23       59       14        414000  22.208908   
2      2187   59:02.5     23       59        2        523000  22.216095   
3      2123   59:02.9     23       59        2        898000  22.228880   
4      2249   59:01.2     23       59        1        234000  21.955398   
5      2388   59:08.6     23       59        8        648000  21.984558   
6      2900   59:02.6     23       59        2        586000  21.306839   
7      2596   59:23.1     23       59       23        148000  21.322042   
8      2210   59:02.1     23       59        2         78000  21.323066   
9      2410   59:08.7     23       59        8        688000  21.155777   
10     2223   59:01.8     23       59        1        773000  19.419499   
11     1944   59:04.6     23       59        4        633000  19.396179   
12     1911   59:03.1    